## Situation and Task
- **Reason**: to provide realtime pricing of bonds with embedded options, prices need to be converted to OAS spread quickly, but the traditional OAS calculator is too slow.
- **Difficulty**: OAS calculator is a high-dimensional input, complex function
- **Existing solution**: pre-compute and cache OAS calc results, but those are expensive.
- **Task**: Build an OAS approximator with fast calculation and guaranteed precision

## Action
- **Initial solution**: Normally people will go for fancy numerical approximation scheme, but I realize machine learning can be applicable here.
    - Usually people take machine learning as a prediction or automated decision scheme. But in fact, it is also a **powerful approximation method: the neural net is even the 'universal approximator'**.
    - Build a machine learning model, trained on the inputs $X$ of the OAS approximator and true output $y$ from the OAS calculator. The trained model can then be the approximator.
- **Further difficulty**: 
    - Find that there are pockets of the input regions where precision is hard to guarantee, and needs more training samples.
    - But training samples are expensive to generate, and what is worse, we do not know these regions before hand. 
- **Refinement**: Made the connection in hyperparameter searching in neural network and solve this above problem adapting from Bayesian optimization/Gaussian process.
	- The problem is similar to **crude oil extraction**: we do not know where the oil is, and it is expensive to dig holes
	- So the solution is that we trial a few places, analyze the soil sample and determine which regions to dig next, and iterate. At the end, if you look at the actual holes we dig in the area, you will find that there are more holes in areas where there is high likelihood of oil.
	- The adapted Bayesian optimization for our problem is similarly an automatic way to search. 
        - As an **iterative process**, given the existing model errors on the current samples, we analyze and determine which region that model is more likely to do poorly (i.e. large model errors so far). Then we sample more in those regions and reevaluate.
        - In this way, we **get away from having to sample too many unnecessary points** for training samples: since for areas where the OAS approximator is already doing a good job, we can afford to sample less, just as we can forget about areas in ground when we are highly certain there is no oil underneath.

## Resolution
- **Achievement**: 
    - Machine learning solution + Bayesian optimization is able to **save 100x compute time and 1000x storage**
    - Make viable a business proposal: it is a **0 to 1 breakthrough**
- **Take-away**. It is also remarkable intellectually, as it showcase the **power of making conceptual connection and adapting solutions**
	- Machine learning algo were used to e.g. identify cats in the picture, but it is just approximating complex logics and functions, which makes it suitable for OAS approximation.
	- Bayesian optimization was traditionally used to find the maximum value of high-dimensional functions, but it can also be used as a systematic way of search, which is suitable to be adapted to sample training data.

## Technical Details

### Explaining OAS calc in general

#### Explain the concept of callable bonds

Callable bonds is just a **straight or vanilla bond + an Bermudan option**. The underlying of the option is the same straight bond. So it is like a bundle of **long stock + selling an Bermudan call** on the same stock.
   - Since option prices cannot be negative, **callable bond price is lower than otherwise identical straight bond**, or **its yield is higher**.
   - **Neither option price nor the straight bond price is observable** for a callable bond, except in the rare case where there is an otherwise identical straight bond that is traded.
   
#### Explain the OAS calculator and why it is so slow

It is an option calculator: given the price of the callable bond, what is the price of the straight bond and the option. 
- Just like pricing an American/Bermudan option via Monte Carlo simulation, we simulate interest-rate paths or **interest-rate trees** to price the straight bond, then on each node, do **backward induction** to account for the fact that the fair value on that node is the minimum value of the strike and the PV of the straight bond.  
- The above process is already time-consuming. On top of that, we also need to **solve for a suitable discount factor so that the discounting of future cashflow, or the PV of the model, equals to callable bond's price.** This is the same as, given the price of a bundle of long stock + short Bermudan call option, figure out what is the price of the stock and the price of the Bermudan call option: the way to go about it is to solve for the stock price so that, for the input volatility, brings about the option so that the theoretical bundle price equal the observed price. **That explains why OAS calculation is expensive**.

### Input space - high dimensional

- price, volatility, coupon, non-call in years, maturity, tenor values of the base curve

- The goal is to approximate the OAS calculator with **guarantee of precision** in a range of the above inputs, 
    - where the range of price, volatility and curve indicate what **market environment** fits the calibration, 
    - while the range of coupon, non-call year and maturity indicate **which bonds can use the approximator**.

- It is clear that a simple gridding and caching approach will quickly run afoul of the **curse of dimensionality**.

- We also tried reducing the dimensionality of **base curve by using Nielson-Siegal components**, i.e. PCA

### The machine learning model - MLP

- We tried single and double-layered MLP, with a `tanh` layer in between.

- **Number of neurons** is found to be sufficient to be 30-50. 
    - The small and simple network **does not require us to spend much time in hyperparameter turning**: empirically, **larger networks have bigger chance of having saddle points**

- We find that `MLPRegressor` in `sklearn` with `lbfgs` as optimizer works reasonably well: `lbfgs` is known to work well for small, simple networks anyways. 
    - Thereby also saving us the efforts to tune hyperparaters of other optimizors, such as `adam`.
    - We find that the default learning rate also works well.

- We specify **No regularization**, as the implicit assumption is the true OAS calculator is not noisy (when its precision is sufficiently high see below).

- Loss function is defined to be **p mean**, where the **larger the p, the more we avert large error**. This is a way to guarantee precision, at least empirically.

- The main headache is the **precision of the true OAS calculator** and to **sample smartly** (see the above point for Gaussian process).
    - The two issues compound each other: to have better precision, it is more expensive to run the true OAS calculator, while it is all the more important to sub-sample that counts.


### The Gaussian process optimization

- In each iteration, train the MLP on the given set of data, obtain the errors
- Train a Gaussian process on the data points of errors: similar to Bayesian optimization, **the Gaussian process is a surrogate model**.
- Further generate some extra points in the input space. See which newly sample points are predicted by the Gaussian process to be significantly non-zero by the Gaussian process. Add those to the set of data to be re-trained by MLP next.
    - We specify significance of zero as just **absolute value of posterior mean over posterior standard deviation exceeding some threshold**: it is a simpliefied version of expected improvement.
    - The feature of Gaussian process ensures that points that are near previously significant points are more likely to get picked at this round. This is due to **Gaussian process's trait that posterior variance is usually small near the sample points, and gradually enlarge when it is farther away**.
    - The above is the **key why this procedure can ensure more samples are drawn in the neighborhood where the current OAS approximation does poorly**.
- Repeat, until there is no added data, or the number of data points reach a threshold

#### Specifying the Gaussian process

- The kernel is **isotropic**, i.e. only depends on the distance between the two input points. 
    - The **distance of input points are weighted Euclidean**, reflecting the belief which dimension the error is likely to be more sensitive on.
    - To customize the above kernel, needed to **override the `__call__` special method in kernel in `sklearn`**.

- We **do not specify extra noise** (i.e. `alpha` is the defaulted very small value in the API). The implicit assumption is we trust the precision of the true OAS calculator we are approximating.

#### Next steps of refining the Gaussian process

- GP is probably reaching the **limit of number of data** (a few thousand) since we need to invert the covariance matrix in training

- GP is probably reaching the **limit of the number of features** as well: should not be more than a dozen.